## this notebooks serves as a test for my model implementation

In [2]:
!pip install /root/git/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
from msspackages import setup_runner
setup_runner(setup_type = 'notebook' , project = 'understanding-eks-data')

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/git/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
msspackages is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
b"Get:1 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]\nHit:2 http://deb.debian.org/debian buster InRelease\nGet:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]\nGet:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [412 kB]\nFetched 504 kB in 1s (811 kB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\n52 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nsudo is already the newest version (1.8.27-1+deb10u4).\n0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.\nReading package lists...\nBui

In [13]:
%pip install hmmlearn
from msspackages import Pyspark_data_ingestion
from pyspark import StorageLevel

from pyspark.sql.functions import *
from hmmlearn import hmm,base
from sklearn.preprocessing import StandardScaler as scale
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
import random
from pyspark.ml.feature import VectorAssembler, StandardScaler

Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.


In [4]:
%%time
test = Pyspark_data_ingestion(year = '2022', month = "7", day = "10", hour ="1", filter_column_value ='Node',setup='128gb')
test_node = test.get_spark()
err, test_df = test.read()

CPU times: user 66.8 ms, sys: 23.7 ms, total: 90.5 ms
Wall time: 20.7 s


## feature selection

In [5]:
features = ['node_cpu_utilization','node_memory_utilization']
#filter inital node df based on request features
node_df =test_df.select("Timestamp", "InstanceId", 'node_cpu_utilization','node_memory_utilization')
node_df = node_df.withColumn("Datetime",(col("Timestamp")/1000).cast("timestamp"))

# Drop NA
cleaned_node_df = node_df.na.drop(subset=features)


#Quality(timestamp filtered) nodes
quality_filtered_node_df = cleaned_node_df.groupBy("InstanceId").agg(count("Timestamp").alias("timestamp_count"))
quality_filtered_nodes = quality_filtered_node_df.filter(col("timestamp_count").between(45,75))

print(3)
#Processed Node DF                                                      
processed_node_df = cleaned_node_df.filter(col("InstanceId").isin(quality_filtered_nodes["InstanceId"]))

3


In [6]:
processed_node_df.show(n=1,vertical=True)

-RECORD 0---------------------------------------
 Timestamp               | 1657425701916        
 InstanceId              | i-06953fcb432461415  
 node_cpu_utilization    | 1.9636783447281012   
 node_memory_utilization | 7.287004490706401    
 Datetime                | 2022-07-10 04:01:... 
only showing top 1 row



In [6]:
## random split data
input_df, test_df = processed_node_df .randomSplit(weights=[0.8,0.2], seed=200)
input_df.show(n=5,vertical=True)

-RECORD 0---------------------------------------
 Timestamp               | 1657415367926        
 InstanceId              | i-0cf25f056cc94aee7  
 node_cpu_utilization    | 0.1968782361829547   
 node_memory_utilization | 1.0431763504720821   
 Datetime                | 2022-07-10 01:09:... 
-RECORD 1---------------------------------------
 Timestamp               | 1657415368207        
 InstanceId              | i-08bd7e60558eb530d  
 node_cpu_utilization    | 1.0533160441504943   
 node_memory_utilization | 7.2665617052198135   
 Datetime                | 2022-07-10 01:09:... 
-RECORD 2---------------------------------------
 Timestamp               | 1657415368693        
 InstanceId              | i-05929ef9891c0ebe1  
 node_cpu_utilization    | 1.9263661848142852   
 node_memory_utilization | 10.366899292369194   
 Datetime                | 2022-07-10 01:09:... 
-RECORD 3---------------------------------------
 Timestamp               | 1657415369173        
 InstanceId         

In [48]:
n = 0
time_steps=12
final_df = None
weight = 2


'''
hour_multiplier = 1
if input_hour=='-1':
    hour_multiplier = 24

    
samplesize = multiplier*1440*weight/time_step
'''
samplesize = 100
features = ['node_cpu_utilization','node_memory_utilization']

In [ ]:
n=0
while n<samplesize:
    ##pick random node
    random_instance = random.choice(input_df.select("InstanceId").rdd.flatMap(list).collect())
    node_df = input_df[(input_df["InstanceId"] ==  random_instance)][["Timestamp", "InstanceId"] + features].select('*')
    node_df = node_df.sort("Timestamp")
    node_df = node_df.na.drop(subset=features)

    #fix negative number bug 
    if node_df.count()-time_steps<= 0:
        print(f'Exception occurred: not enough data')
        continue


    #standardize data from the node
    assembler = VectorAssembler(inputCols=features, outputCol="vectorized_features")
    scaler = StandardScaler(inputCol = "vectorized_features", outputCol = "scaled_features", withMean=True, withStd=True)
    pipeline = Pipeline(stages=[assembler, scaler])
    node_df = pipeline.fit(node_df).transform(node_df)

    #pick random time slice of 12 timestamps from this node
    start = random.choice(range(node_df.count()-time_steps))
    node_slice_df = node_df.withColumn('rn', row_number().over(Window.orderBy("Timestamp"))).filter((col("rn") >= start) & (col("rn") < start+time_steps)).select(["Timestamp",'scaled_features'])
    node_slice_df = node_slice_df.select('Timestamp','scaled_features')



    #fill the large dataset
    if not final_df:
        final_df = node_slice_df
    else:
        final_df = final_df.union( node_slice_df)

    print(f'Finished with sample #{n}')
    n+=1

Finished with sample #0
Finished with sample #1
Finished with sample #2
Finished with sample #3
Finished with sample #4
Finished with sample #5


In [ ]:
final_df.groupBy("Timestamp").agg(mean("scaled_features").alias("scaled_features")).show()



In [47]:
final_df.show(n=1)

+-------------+
|    Timestamp|
+-------------+
|1657417441081|
+-------------+
only showing top 1 row



In [ ]:
## using pandas to clean my codes instead